# Capstone Project - The Battle of the Neighborhoods (Week 1)
### Applied Data Science Capstone by IBM/Coursera

## Introduction: Business Problem <a name="introduction"></a>

¿Where is the best location for a Specialty Coffee Shop in Santiado of Chile?

For this project we will be targeted to stakeholders interested in opening an Specialty Coffee Shop in Santiago, Chile.

Coffee is a fundamental part of food culture around the world and the specialty coffee is a growing market. 

Coffee consumption has increased dramatically and the consumers in general are moving toward higher-quality, gourmet coffee. For years, you could only get coffee beans and grounds from a few companies and regions, but the foodservice trend toward higher-quality and craft foods has opened up many other options. Nowadays, there is a nearly endless selection of coffee from all over the world that wouldn't have been available a few decades ago.

The age of coffee consumers also has decreased. The change was identified through the findings of a 2017 study on coffee consumption. The study was published in the journal Annals of Internal Medicine and involved more than 520,000 people. Their findings showed that drinking coffee regularly could improve life expectancy. After these results were published, the National Coffee Association (NCA) discovered that the percentage of daily coffee drinkers increased from 57% to 62% in the course of a year. In the United States alone, 44% of those coffee consumers are millennials. (https://www.webstaurantstore.com/blog/2356/top-coffee-trends.html)

There are 941 coffee shops in the metropolitan area of Santiago and this number has been increasing in the last 10 years. (http://santiagocoffeelovers.cl/cafeterias-especialidad-nuevo-mundo-sabores-experiencias/)

We will explore the metropolitan area of Santiago to find the best locations based on a good quality of life with less number of coffee shops and places frequented by young people.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing coffee shops in the areas explored
* venues like museums, art gallerys, cultural centers or farmer markets (places frecuented by young people)
* a zone with a good quality of life

Following data sources will be needed to extract/generate the required information:
* List of the Zones of Santiago using **wikipedia** https://es.wikipedia.org/wiki/Anexo:Comunas_de_Santiago_de_Chile

<img src="https://upload.wikimedia.org/wikipedia/commons/a/af/Satellite_image_of_Santiago%2C_Chile_-_October_24%2C_2014.jpg" style="height:600px">

* Population data of Santiago also of the same **wikipedia** page
<img src="https://upload.wikimedia.org/wikipedia/commons/d/de/Densidad_poblacional_de_Santiago_de_Chile_en_2017.jpg" style="height:400px">


* **HERE geocode** to obtain the latitude and longitude of the zones (communes) of Santiago
* number and location of coffee shops using **Foursquare API** (https://developer.foursquare.com/docs/api/venues/explore)
* venues by zone and their categories using **Foursquare API** (https://developer.foursquare.com/docs/api/venues/explore)